In [1]:
from oauth2client.service_account import ServiceAccountCredentials
from bs4 import BeautifulSoup

import json
import requests
import gspread
import datetime, time
import os

In [2]:
def getJSON(exch,url):
    start_time = time.time()
    header = {'x-requested-with': 'XMLHttpRequest'}
    mainPage = requests.get(url, headers = header)
    data = mainPage.json()
    #print('%s API Query finished in %s seconds') % (exch, str(round(time.time() - start_time,2)))
    return data

def getValue(data,layer):
    if layer in ('IGNORE'):
        return 0
    else:
        a = data
        alist = layer.split(",")
        for i in xrange(len(alist)):
            a = a.get(alist[i])

        if isinstance(a,list):
            a = a[0]
        return a

def getArbPrem(dict_pair):
    min_exch = min(dict_pair.keys(), key=(lambda k: dict_pair[k]))
    max_exch = max(dict_pair.keys(), key=(lambda k: dict_pair[k]))
    return dict_pair[max_exch] / dict_pair[min_exch]

def getArbDir(dict_pair):
    min_exch = min(dict_pair.keys(), key=(lambda k: dict_pair[k]))
    max_exch = max(dict_pair.keys(), key=(lambda k: dict_pair[k]))
    return min_exch + ' -> ' + max_exch

In [3]:
#Get USD/KRW Spot Rate
currency_layer = 'http://www.apilayer.net/api/live?access_key=5a76ffa1f274620e53f1536251fa7201&format=1'
usd_krw = getValue(getJSON('Currency Layer',currency_layer),'quotes,USDKRW')

In [36]:
#XRP Price URL
coinone_ticker_xrp  = 'https://api.coinone.co.kr/ticker/?currency=xrp&format=json'
korbit_ticker_xrp   = 'https://api.korbit.co.kr/v1/ticker/detailed?currency_pair=xrp_krw'
bithumb_ticker_xrp  = 'https://api.bithumb.com/public/ticker/xrp'
bitfinex_ticker_xrp = 'https://api.bitfinex.com/v1/pubticker/xrpusd'
poloniex_ticker_xrp = 'https://poloniex.com/public?command=returnTicker&currencyPair=ALL'
bitstamp_ticker_xrp = 'https://www.bitstamp.net/api/v2/ticker/xrpusd/'

#BTC Price URL
coinone_ticker_btc  = 'https://api.coinone.co.kr/ticker/?currency=btc&format=json'
korbit_ticker_btc   = 'https://api.korbit.co.kr/v1/ticker/detailed?currency_pair=btc_krw'
bithumb_ticker_btc  = 'https://api.bithumb.com/public/ticker/btc'
bitfinex_ticker_btc = 'https://api.bitfinex.com/v1/pubticker/btcusd'
gemini_ticker_btc   = 'https://api.gemini.com/v1/pubticker/btcusd'
gdax_ticker_btc     = 'https://api.gdax.com/products/btc-usd/ticker'
bitstamp_ticker_btc = 'https://www.bitstamp.net/api/v2/ticker/btcusd/'

#LTC Price URL
coinone_ticker_ltc  = 'https://api.coinone.co.kr/ticker/?currency=ltc&format=json'
bithumb_ticker_ltc  = 'https://api.bithumb.com/public/ticker/ltc'
bitfinex_ticker_ltc = 'https://api.bitfinex.com/v1/pubticker/ltcusd'
gemini_ticker_ltc   = 'https://api.gemini.com/v1/pubticker/ltcusd'
gdax_ticker_ltc     = 'https://api.gdax.com/products/ltc-usd/ticker'
bitstamp_ticker_ltc = 'https://www.bitstamp.net/api/v2/ticker/ltcusd/'

#ETH Price URL
coinone_ticker_eth  = 'https://api.coinone.co.kr/ticker/?currency=eth&format=json'
korbit_ticker_eth   = 'https://api.korbit.co.kr/v1/ticker/detailed?currency_pair=eth_krw'
bithumb_ticker_eth  = 'https://api.bithumb.com/public/ticker/eth'
bitfinex_ticker_eth = 'https://api.bitfinex.com/v1/pubticker/ethusd'
gemini_ticker_eth   = 'https://api.gemini.com/v1/pubticker/ethusd'
gdax_ticker_eth     = 'https://api.gdax.com/products/eth-usd/ticker'
bitstamp_ticker_eth = 'https://www.bitstamp.net/api/v2/ticker/ethusd/'

#ETH/XRP Price URL
bittrex_ticker_ethxrp = 'https://bittrex.com/api/v1.1/public/getmarketsummary?market=eth-xrp'
binance_ticker_ethxrp = 'https://api.binance.com/api/v1/ticker/24hr?symbol=XRPETH'

#BTC/ETH Price URL
bitfinex_ticker_btceth = 'https://api.bitfinex.com/v1/pubticker/ethbtc'
gemini_ticker_btceth   = 'https://api.gemini.com/v1/pubticker/ethbtc'
gdax_ticker_btceth     = 'https://api.gdax.com/products/eth-btc/ticker'
bittrex_ticker_btceth  = 'https://bittrex.com/api/v1.1/public/getmarketsummary?market=btc-eth'
bitstamp_ticker_btceth = 'https://www.bitstamp.net/api/v2/ticker/ethbtc/'
binance_ticker_btceth  = 'https://api.binance.com/api/v1/ticker/24hr?symbol=ETHBTC'

#BTC/XRP Price URL
bitfinex_ticker_btcxrp = 'https://api.bitfinex.com/v1/pubticker/xrpbtc'
bittrex_ticker_btcxrp = 'https://bittrex.com/api/v1.1/public/getmarketsummary?market=btc-xrp'
bitstamp_ticker_btcxrp = 'https://www.bitstamp.net/api/v2/ticker/xrpbtc/'
binance_ticker_btcxrp  = 'https://api.binance.com/api/v1/ticker/24hr?symbol=XRPBTC'

#LTC/BTC Price URL
binance_ticker_ltcbtc = 'https://api.binance.com/api/v1/ticker/24hr?symbol=LTCBTC'

In [37]:
#KRW Pairs
xrp_dict = {  'CoinOne'  : float(getValue(getJSON('CoinOne',coinone_ticker_xrp),'last')) / usd_krw
             ,'Korbit'   : float(getValue(getJSON('Korbit',korbit_ticker_xrp),'last')) / usd_krw
             ,'Bithumb'  : float(getValue(getJSON('Bithumb',bithumb_ticker_xrp),'data,sell_price')) / usd_krw
             ,'Bitfinex' : float(getValue(getJSON('Bitfinex',bitfinex_ticker_xrp),'last_price'))
             ,'Poloniex' : float(getValue(getJSON('Poloniex',poloniex_ticker_xrp),'USDT_XRP,last'))
             ,'Bitstamp' : float(getValue(getJSON('Bitstamp',bitstamp_ticker_xrp),'last'))
           }

btc_dict = {  'CoinOne'  : float(getValue(getJSON('CoinOne',coinone_ticker_btc),'last')) / usd_krw
             ,'Korbit'   : float(getValue(getJSON('Korbit',korbit_ticker_btc),'last')) / usd_krw
             ,'Bithumb'  : float(getValue(getJSON('Bithumb',bithumb_ticker_btc),'data,sell_price')) / usd_krw
             ,'Bitfinex' : float(getValue(getJSON('Bitfinex',bitfinex_ticker_btc),'last_price'))
             ,'Poloniex' : float(getValue(getJSON('Poloniex',poloniex_ticker_xrp),'USDT_BTC,last'))
             ,'Gemini'   : float(getValue(getJSON('Gemini',gemini_ticker_btc),'last'))
             ,'GDAX'     : float(getValue(getJSON('GDAX',gdax_ticker_btc),'ask'))
             ,'Bitstamp' : float(getValue(getJSON('Bitstamp',bitstamp_ticker_btc),'last'))
           }

ltc_dict = {  'CoinOne'  : float(getValue(getJSON('CoinOne',coinone_ticker_ltc),'last')) / usd_krw
             ,'Bithumb'  : float(getValue(getJSON('Bithumb',bithumb_ticker_ltc),'data,sell_price')) / usd_krw
             ,'Bitfinex' : float(getValue(getJSON('Bitfinex',bitfinex_ticker_ltc),'last_price'))
             ,'Poloniex' : float(getValue(getJSON('Poloniex',poloniex_ticker_xrp),'USDT_LTC,last'))
             ,'GDAX'     : float(getValue(getJSON('GDAX',gdax_ticker_ltc),'ask'))
             ,'Bitstamp' : float(getValue(getJSON('Bitstamp',bitstamp_ticker_ltc),'last'))
           }

eth_dict = {  'CoinOne'  : float(getValue(getJSON('CoinOne',coinone_ticker_eth),'last')) / usd_krw
             ,'Korbit'   : float(getValue(getJSON('Korbit',korbit_ticker_eth),'last')) / usd_krw
             ,'Bithumb'  : float(getValue(getJSON('Bithumb',bithumb_ticker_eth),'data,sell_price')) / usd_krw
             ,'Bitfinex' : float(getValue(getJSON('Bitfinex',bitfinex_ticker_eth),'last_price'))
             ,'Poloniex' : float(getValue(getJSON('Poloniex',poloniex_ticker_xrp),'USDT_ETH,last'))
             ,'Gemini'   : float(getValue(getJSON('Gemini',gemini_ticker_eth),'last'))
             ,'GDAX'     : float(getValue(getJSON('GDAX',gdax_ticker_eth),'ask'))
             ,'Bitstamp' : float(getValue(getJSON('Bitstamp',bitstamp_ticker_eth),'last'))
           }

In [45]:
#Arb Crosses
btc_eth_dict = { 'Bitfinex' : float(getValue(getJSON('Bitfinex',bitfinex_ticker_btceth),'last_price'))
                ,'Poloniex' : float(getValue(getJSON('Poloniex',poloniex_ticker_xrp),'BTC_ETH,last'))
                ,'Gemini'   : float(getValue(getJSON('Gemini',gemini_ticker_btceth),'last'))
                ,'GDAX'     : float(getValue(getJSON('GDAX',gdax_ticker_btceth),'ask'))
                ,'Coinone'  : float(eth_dict['CoinOne'] / btc_dict['CoinOne'])
                ,'Bithumb'  : float(eth_dict['Bithumb'] / btc_dict['Bithumb'])
                ,'Korbit'   : float(eth_dict['Korbit']  / btc_dict['Korbit'])
                ,'Bittrex'  : float(getJSON('Bittrex',bittrex_ticker_btceth)['result'][0]['Last'])
                ,'Bitstamp' : float(getValue(getJSON('Bitstamp',bitstamp_ticker_btceth),'last'))
                ,'BitstampX': float(eth_dict['Bitstamp']  / btc_dict['Bitstamp'])
                ,'Binance'  : float(getValue(getJSON('Binance',binance_ticker_btceth),'lastPrice'))
               }

btc_xrp_dict = { 'Bitfinex'  : float(getValue(getJSON('Bitfinex',bitfinex_ticker_btcxrp),'last_price'))
                ,'Poloniex'  : float(getValue(getJSON('Poloniex',poloniex_ticker_xrp),'BTC_XRP,last'))
                ,'Coinone'   : float(xrp_dict['CoinOne'] / btc_dict['CoinOne'])
                ,'Bithumb'   : float(xrp_dict['Bithumb'] / btc_dict['Bithumb'])
                ,'Korbit'    : float(xrp_dict['Korbit']  / btc_dict['Korbit'])
                ,'Bittrex'   : float(getJSON('Bittrex',bittrex_ticker_btcxrp)['result'][0]['Last'])
                ,'Bitstamp'  : float(getValue(getJSON('Bitstamp',bitstamp_ticker_btcxrp),'last'))
                ,'BitstampX' : float(xrp_dict['Bitstamp']  / btc_dict['Bitstamp'])
                ,'Binance'   : float(getValue(getJSON('Binance',binance_ticker_btcxrp),'lastPrice'))
               }

ltc_xrp_dict = { 'Bitfinex' : float(xrp_dict['Bitfinex'] / ltc_dict['Bitfinex'])
                ,'Poloniex' : float(xrp_dict['Poloniex'] / ltc_dict['Poloniex'])
                ,'Coinone'  : float(xrp_dict['CoinOne']  / ltc_dict['CoinOne'])
                ,'Bithumb'  : float(xrp_dict['Bithumb']  / ltc_dict['Bithumb'])
                ,'BitstampX': float(xrp_dict['Bitstamp'] / ltc_dict['Bitstamp'])
               }

ltc_btc_dict = { 'Bitfinex' : float(btc_dict['Bitfinex'] / ltc_dict['Bitfinex'])
                ,'Poloniex' : float(btc_dict['Poloniex'] / ltc_dict['Poloniex'])
                ,'Coinone'  : float(btc_dict['CoinOne']  / ltc_dict['CoinOne'])
                ,'Bithumb'  : float(btc_dict['Bithumb']  / ltc_dict['Bithumb'])
                ,'BitstampX': float(btc_dict['Bitstamp'] / ltc_dict['Bitstamp'])
                ,'Binance'   : 1/float(getValue(getJSON('Binance',binance_ticker_ltcbtc),'lastPrice'))                
               }

eth_xrp_dict = { 'Coinone'  : float(xrp_dict['CoinOne']  / eth_dict['CoinOne'])
                ,'Korbit'   : float(xrp_dict['Korbit']   / eth_dict['Korbit'])
                ,'Bithumb'  : float(xrp_dict['Bithumb']  / eth_dict['Bithumb'])
                ,'Bitfinex' : float(xrp_dict['Bitfinex'] / eth_dict['Bitfinex'])
                ,'Poloniex' : float(xrp_dict['Poloniex'] / eth_dict['Poloniex'])
                ,'Bittrex'  : float(getJSON('Bittrex',bittrex_ticker_ethxrp)['result'][0]['Last'])
                ,'Bitstamp' : float(xrp_dict['Bitstamp'] / eth_dict['Bitstamp'])
                ,'Binance'   : float(getValue(getJSON('Binance',binance_ticker_ethxrp),'lastPrice'))
               }

In [46]:
print 'BTC -> ETH : ' + getArbDir(btc_eth_dict)
print round(getArbPrem(btc_eth_dict),5)

BTC -> ETH : Gemini -> Bithumb
1.04695


In [47]:
print 'XRP -> BTC : ' + getArbDir(btc_xrp_dict)
print round(getArbPrem(btc_xrp_dict),5)

XRP -> BTC : Korbit -> Bithumb
1.05724


In [48]:
print 'XRP -> ETH : ' + getArbDir(eth_xrp_dict)
print round(getArbPrem(eth_xrp_dict),5)

XRP -> ETH : Korbit -> Binance
1.04527


In [49]:
print 'LTC -> XRP : ' + getArbDir(ltc_xrp_dict)
print round(getArbPrem(ltc_xrp_dict),5)

LTC -> XRP : Coinone -> Bithumb
1.00661


In [50]:
print 'LTC -> BTC : ' + getArbDir(ltc_btc_dict)
print round(getArbPrem(ltc_btc_dict),5)

LTC -> BTC : Bithumb -> Binance
1.04103
